In [5]:
# pip install aiohttp 필요
import asyncio
from typing import Iterable, Dict
import aiohttp


async def fetch_all(
    urls: Iterable[str],
    *,
    concurrency: int = 5,  # 동시에 보낼 최대 요청 수
    timeout: float = 10.0,  # 각 요청의 총 타임아웃(초)
    retries: int = 2,  # 실패 시 재시도 횟수
) -> Dict[str, str]:
    """
    여러 URL을 비동기적으로 가져와 {url: 응답본문 또는 에러메시지}를 반환합니다.

    - concurrency: 세마포어로 동시성 제한
    - timeout: aiohttp.ClientTimeout(total=timeout)
    - retries: 지수 백오프(0.5s, 1s, 2s ...)로 재시도
    """
    results: Dict[str, str] = {}
    sem = asyncio.Semaphore(concurrency)  # 동시 요청 수 제한

    # 세션은 가능한 한 재사용(커넥션 풀)하는 것이 효율적
    timeout_cfg = aiohttp.ClientTimeout(total=timeout)
    async with aiohttp.ClientSession(timeout=timeout_cfg) as session:
        print("a")

        async def one(url: str) -> None:
            # 내부 헬퍼 코루틴(외부에 노출되는 함수는 fetch_all 하나뿐)
            for attempt in range(retries + 1):
                try:
                    async with sem:  # 동시성 제한 구간
                        async with session.get(url) as resp:
                            resp.raise_for_status()  # 4xx/5xx면 예외
                            results[url] = await resp.text()  # 본문 수집
                            return
                except Exception as e:
                    # 마지막 시도 전에는 지수 백오프로 대기 후 재시도
                    if attempt < retries:
                        await asyncio.sleep(0.5 * (2**attempt))
                    else:
                        # 모든 시도 실패 시 에러 메시지를 결과에 저장
                        results[url] = f"ERROR: {type(e).__name__}: {e}"

        # 모든 작업을 동시에 스케줄링하고 완료를 대기
        print("b")
        await asyncio.gather(*(one(u) for u in urls))

    return results


In [6]:
test_urls = [
    "https://example.com",
    "https://httpbin.org/get",
    "https://httpbin.org/status/404",  # 일부러 에러 테스트
]
data = await fetch_all(test_urls, concurrency=10, timeout=5.0, retries=2)
for u, body in data.items():
    print(u, "=>", (body[:80] + "...") if len(body) > 80 else body)

a
b
https://example.com => <!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta chars...
https://httpbin.org/get => {
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gz...
https://httpbin.org/status/404 => ERROR: ClientResponseError: 404, message='NOT FOUND', url='https://httpbin.org/s...
